# Cache

Nginx provides a caching facility that saves responses from proxied URLs and reuses them later. You can enable and configure it using directives that start with `proxy_cache`. There is a special tutorial on the Nginx website that covers Nginx configuration.

There is [a guide for caching](https://blog.nginx.org/blog/nginx-caching-guide#:~:text=The%20proxy_cache_path%20directive%20sets%20the,the%20proxy_cache%20directive%20activates%20it.&text=The%20parameters%20to%20the%20proxy_cache_path,path%2Fto%2Fcache%2F.) at official nginx documentation.

In the next cell, the setup defined earlier is executed, and we access the root page.

In [45]:
docker compose -f ../reverse_proxy_files/docker-compose.yml up -d &> /dev/null

Don't forget to clean up the environment after everything is done.

In [104]:
docker compose -f ../reverse_proxy_files/docker-compose.yml down &> /dev/null

## Expire time

You can regulate expiration type through the `proxy_cache_valid` directive. Specify status codes and their expiration times within the `location` directive.

---

The following cell configures Nginx to expire cached responses with a 200 status code after 5 seconds:

In [88]:
docker exec -i experiment_nginx sh -c 'cat >  /etc/nginx/nginx.conf' << EOF
events {}
http {

    proxy_cache_path /var/cache/nginx/proxy_cache keys_zone=my_cache:10m inactive=3s;

    server {
        listen 80;
        location / {
            proxy_cache my_cache;
            proxy_cache_valid 200 5s;
            proxy_pass http://client_container/bytes/50;
        }
    }
}
EOF

docker exec -it experiment_nginx nginx -s reload

2024/09/09 12:57:51 [notice] 130#130: signal process started


Now let's check if it works—the following cell sends a request to Nginx every second for some time. After a few seconds, the output changes, indicating that the cache has expired and a new response is generated.

In [103]:
for i in {1..10}; do
    echo $(curl -s localhost:80/ | tr -d '\0')
    sleep 1
done

>�E�V��9���`��Wè�o6F��A(ف��/�^f�{)�2�'��`�]�)��
>�E�V��9���`��Wè�o6F��A(ف��/�^f�{)�2�'��`�]�)��
>�E�V��9���`��Wè�o6F��A(ف��/�^f�{)�2�'��`�]�)��
>�E�V��9���`��Wè�o6F��A(ف��/�^f�{)�2�'��`�]�)��
>�E�V��9���`��Wè�o6F��A(ف��/�^f�{)�2�'��`�]�)��
>�E�V��9���`��Wè�o6F��A(ف��/�^f�{)�2�'��`�]�)��
��͹�Fg��-�4P@���BY�b����Q��ψv�P����6٘�c��r
��͹�Fg��-�4P@���BY�b����Q��ψv�P����6٘�c��r
��͹�Fg��-�4P@���BY�b����Q��ψv�P����6٘�c��r
��͹�Fg��-�4P@���BY�b����Q��ψv�P����6٘�c��r
